In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cp "/content/drive/MyDrive/DATS 5980/aokvqa.zip" /content/

In [3]:
%cp "/content/drive/MyDrive/DATS 5980/coco_filtered.zip" /content/

In [4]:
%cd /content/

/content


In [5]:
!unzip /content/aokvqa.zip

Archive:  /content/aokvqa.zip
   creating: aokvqa/
  inflating: aokvqa/aokvqa_v1p0_test.json  
  inflating: aokvqa/aokvqa_v1p0_train.json  
  inflating: aokvqa/aokvqa_v1p0_val.json  
  inflating: aokvqa/large_vocab_train.csv  
  inflating: aokvqa/specialized_vocab_train.csv  


In [6]:
!unzip /content/coco_filtered.zip

Streaming output truncated to the last 5000 lines.
  inflating: coco_filtered/test2017/000000201802.jpg  
  inflating: coco_filtered/test2017/000000201812.jpg  
  inflating: coco_filtered/test2017/000000201975.jpg  
  inflating: coco_filtered/test2017/000000202014.jpg  
  inflating: coco_filtered/test2017/000000202038.jpg  
  inflating: coco_filtered/test2017/000000202208.jpg  
  inflating: coco_filtered/test2017/000000202294.jpg  
  inflating: coco_filtered/test2017/000000202314.jpg  
  inflating: coco_filtered/test2017/000000202435.jpg  
  inflating: coco_filtered/test2017/000000202555.jpg  
  inflating: coco_filtered/test2017/000000202625.jpg  
  inflating: coco_filtered/test2017/000000202758.jpg  
  inflating: coco_filtered/test2017/000000202771.jpg  
  inflating: coco_filtered/test2017/000000202930.jpg  
  inflating: coco_filtered/test2017/000000203394.jpg  
  inflating: coco_filtered/test2017/000000203508.jpg  
  inflating: coco_filtered/test2017/000000203550.jpg  
  inflating: c

In [7]:
import os
import json
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image

In [8]:
import random
import numpy as np
import torch

def set_seed(seed: int = 42):
    """
    Set random seed for reproducibility across Python, NumPy, and PyTorch.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # For deterministic behavior (slower but fully reproducible)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [9]:
aokvqa_dir = os.getenv('AOKVQA_DIR', "/content/aokvqa")
coco_filtered_dir = os.getenv('COCO_FILTERED_DIR', r"/content/coco_filtered")

In [10]:
def load_aokvqa(aokvqa_dir, split, version='v1p0'):
    assert split in ['train', 'val', 'test', 'test_w_ans']
    dataset = json.load(open(
        os.path.join(aokvqa_dir, f"aokvqa_{version}_{split}.json")
    ))
    return dataset

def get_coco_path(split, image_id, coco_dir):
    return os.path.join(coco_dir, f"{split}2017", f"{image_id:012}.jpg")


In [11]:
val_aokvqa_dataset = load_aokvqa(aokvqa_dir, 'val')
train_aokvqa_dataset = load_aokvqa(aokvqa_dir, 'train')
test_aokvqa_dataset = load_aokvqa(aokvqa_dir, 'test')

In [12]:
print(f"Train dataset size: {len(train_aokvqa_dataset)}")
print(f"Validation dataset size: {len(val_aokvqa_dataset)}")
print(f"Test dataset size: {len(test_aokvqa_dataset)}")

Train dataset size: 17056
Validation dataset size: 1145
Test dataset size: 6702


In [13]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

Using device: cuda


In [14]:
import torch
from transformers import AutoModelForCausalLM

from deepseek_vl.models import DeepseekVLV2Processor, DeepseekVLV2ForCausalLM
from deepseek_vl.utils.io import load_pil_images

ModuleNotFoundError: No module named 'deepseek_vl'

In [ ]:
# specify the path to the model
model_path = "deepseek-ai/deepseek-vl2-small"
vl_chat_processor: DeepseekVLV2Processor = DeepseekVLV2Processor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: DeepseekVLV2ForCausalLM = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

In [ ]:
# model_name = "HuggingFaceTB/SmolVLM-256M-Instruct"
# model_name = "HuggingFaceTB/SmolVLM-500M-Instruct"
model_name = "HuggingFaceTB/SmolVLM-Instruct"

# model_name = "HuggingFaceTB/SmolVLM-256M-Base"

Single Prediction

In [ ]:
# Initialize processor and model
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForVision2Seq.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    # _attn_implementation="flash_attention_2" if DEVICE == "cuda" else "eager",
).to(DEVICE)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/429 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/486 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/modeling_auto.py:2284: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [ ]:
def get_prediction(data_point, coco_dir, split):
    question = data_point['question']

    image_path = get_coco_path(split, data_point['image_id'], coco_dir)
    # image = load_image(image_path)
    image = Image.open(image_path).convert("RGB")

    # Create input messages
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": question}
            ]
        },
    ]

    # Prepare inputs
    prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(text=prompt, images=[image], return_tensors="pt")
    inputs = inputs.to(DEVICE)

    # Generate outputs
    generated_ids = model.generate(**inputs, max_new_tokens=500)
    generated_texts = processor.batch_decode(
        generated_ids,
        skip_special_tokens=True,
    )
    return data_point, generated_texts[0]


In [ ]:
predictions = []
for idx, data_point in enumerate(val_aokvqa_dataset):
    data_point, pred = get_prediction(data_point, coco_filtered_dir, 'val')
    print(idx, end=",")
    predictions.append((data_point, pred))
    break

0,

In [ ]:
data_point, pred

({'split': 'val',
  'image_id': 461751,
  'question_id': '22jbM6gDxdaMaunuzgrsBB',
  'question': "What is in the motorcyclist's mouth?",
  'choices': ['toothpick', 'food', 'popsicle stick', 'cigarette'],
  'correct_choice_idx': 3,
  'direct_answers': ['cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette'],
  'difficult_direct_answer': False,
  'rationales': ["He's smoking while riding.",
   'The motorcyclist has a lit cigarette in his mouth while he rides on the street.',
   'The man is smoking.']},
 "User:\n\n\n\n\nWhat is in the motorcyclist's mouth?\nAssistant: The motorcyclist is wearing a mouth guard.")

batch

In [ ]:
# model_name = "HuggingFaceTB/SmolVLM-256M-Instruct"

# Initialize processor and model
processor = AutoProcessor.from_pretrained(model_name)

# Force left padding for decoder-only generation
if hasattr(processor, "tokenizer"):
    processor.tokenizer.padding_side = "left"
    if processor.tokenizer.pad_token is None:
        processor.tokenizer.pad_token = processor.tokenizer.eos_token

model = AutoModelForVision2Seq.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    # _attn_implementation="flash_attention_2" if DEVICE == "cuda" else "eager",
).to(DEVICE)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/429 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/486 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/modeling_auto.py:2284: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/4.49G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [ ]:
def get_batch_predictions(data_points, coco_dir, split):
    """
    Generate predictions for a batch of data points.

    Args:
        data_points (list): List of data point dicts.
        coco_dir (str): Directory containing COCO images.
        split (str): Dataset split ('train', 'val', 'test').

    Returns:
        List of (data_point, prediction) tuples.
    """
    # Prepare images and questions
    images = []
    batch_messages = []
    for data_point in data_points:
        question = data_point['question']
        choices = data_point['choices']

        mcq_question = f"Question: {question}. These are the choices: {choices}. Select the right choice. Answer:"

        image_path = get_coco_path(split, data_point['image_id'], coco_dir)
        image = Image.open(image_path).convert("RGB")
        images.append(image)
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": mcq_question}
                ]
            }
        ]
        batch_messages.append(messages)

    # Prepare prompts for each message
    batch_prompts = [processor.apply_chat_template(msg, add_generation_prompt=True) for msg in batch_messages]

    # Prepare inputs for the batch
    batch_inputs = processor(
        text=batch_prompts,
        images=images,
        return_tensors="pt",
        padding=True
    )
    batch_inputs = batch_inputs.to(DEVICE)

    # Generate outputs
    batch_generated_ids = model.generate(**batch_inputs)
    batch_generated_texts = processor.batch_decode(
        batch_generated_ids,
        skip_special_tokens=True,
    )

    return list(zip(data_points, batch_generated_texts))

In [ ]:
def run_batch_predictions(aokvqa_dataset, batch_size, coco_dir, split):
    predictions = []
    for i in range(0, len(aokvqa_dataset), batch_size):
        batch = aokvqa_dataset[i:i+batch_size]
        batch_results = get_batch_predictions(batch, coco_filtered_dir, split)
        for item in batch_results:
            predictions.append(item)
        print(f"Batch number: {(i+1)//batch_size}. Progress: {(i+1) / len(aokvqa_dataset)*100:.2f}%")
    return predictions

In [ ]:
batch_size = 4
dataset = val_aokvqa_dataset[: 150]

predictions = run_batch_predictions(dataset, batch_size, coco_filtered_dir, 'val')

Batch number: 0. Progress: 0.67%
Batch number: 1. Progress: 3.33%
Batch number: 2. Progress: 6.00%
Batch number: 3. Progress: 8.67%
Batch number: 4. Progress: 11.33%
Batch number: 5. Progress: 14.00%
Batch number: 6. Progress: 16.67%
Batch number: 7. Progress: 19.33%
Batch number: 8. Progress: 22.00%
Batch number: 9. Progress: 24.67%
Batch number: 10. Progress: 27.33%
Batch number: 11. Progress: 30.00%
Batch number: 12. Progress: 32.67%
Batch number: 13. Progress: 35.33%
Batch number: 14. Progress: 38.00%
Batch number: 15. Progress: 40.67%
Batch number: 16. Progress: 43.33%
Batch number: 17. Progress: 46.00%
Batch number: 18. Progress: 48.67%
Batch number: 19. Progress: 51.33%
Batch number: 20. Progress: 54.00%
Batch number: 21. Progress: 56.67%
Batch number: 22. Progress: 59.33%
Batch number: 23. Progress: 62.00%
Batch number: 24. Progress: 64.67%
Batch number: 25. Progress: 67.33%
Batch number: 26. Progress: 70.00%
Batch number: 27. Progress: 72.67%
Batch number: 28. Progress: 75.33%

In [ ]:
len(predictions)

150

In [ ]:
import pickle

model_name_mod = model_name.split("/")[1]
output_path = f"/content/drive/MyDrive/DATS 5980/val_predictions_mcq_{model_name_mod}.pkl"

with open(output_path, 'wb') as fp:
    pickle.dump(predictions, fp)

print(f"Predictions saved to {output_path}")

Predictions saved to /content/drive/MyDrive/DATS 5980/val_predictions_mcq_SmolVLM-Instruct.pkl


In [ ]:
predictions[0]

({'split': 'val',
  'image_id': 461751,
  'question_id': '22jbM6gDxdaMaunuzgrsBB',
  'question': "What is in the motorcyclist's mouth?",
  'choices': ['toothpick', 'food', 'popsicle stick', 'cigarette'],
  'correct_choice_idx': 3,
  'direct_answers': ['cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette',
   'cigarette'],
  'difficult_direct_answer': False,
  'rationales': ["He's smoking while riding.",
   'The motorcyclist has a lit cigarette in his mouth while he rides on the street.',
   'The man is smoking.']},
 "User:\n\n\n\n\nQuestion: What is in the motorcyclist's mouth?. These are the choices: ['toothpick', 'food', 'popsicle stick', 'cigarette']. Select the right choice. Answer:\nAssistant: ['cigarette'].")

In [ ]:
batch_size = 4
predictions = []
dataset = test_aokvqa_dataset[: 150]
# dataset = test_aokvqa_dataset[:2400]
# dataset = test_aokvqa_dataset[2400: 3600]
# dataset = test_aokvqa_dataset[3600: 5000]
# dataset = test_aokvqa_dataset[5000:]

print(f"dataset size: {len(dataset)}")

predictions = run_batch_predictions(dataset, batch_size, coco_filtered_dir, 'test')

dataset size: 6702
Batch number: 0. Progress: 0.67%
Batch number: 1. Progress: 3.33%
Batch number: 2. Progress: 6.00%
Batch number: 3. Progress: 8.67%
Batch number: 4. Progress: 11.33%
Batch number: 5. Progress: 14.00%
Batch number: 6. Progress: 16.67%
Batch number: 7. Progress: 19.33%
Batch number: 8. Progress: 22.00%
Batch number: 9. Progress: 24.67%
Batch number: 10. Progress: 27.33%
Batch number: 11. Progress: 30.00%
Batch number: 12. Progress: 32.67%
Batch number: 13. Progress: 35.33%
Batch number: 14. Progress: 38.00%
Batch number: 15. Progress: 40.67%
Batch number: 16. Progress: 43.33%
Batch number: 17. Progress: 46.00%
Batch number: 18. Progress: 48.67%
Batch number: 19. Progress: 51.33%
Batch number: 20. Progress: 54.00%
Batch number: 21. Progress: 56.67%
Batch number: 22. Progress: 59.33%
Batch number: 23. Progress: 62.00%
Batch number: 24. Progress: 64.67%
Batch number: 25. Progress: 67.33%
Batch number: 26. Progress: 70.00%
Batch number: 27. Progress: 72.67%
Batch number: 2

In [ ]:
len(predictions)

150

In [ ]:
import pickle

output_path = '/content/drive/MyDrive/DATS 5980/test_predictions_mcq.pkl'
with open(output_path, 'wb') as fp:
    pickle.dump(predictions, fp)

print(f"Predictions saved to {output_path}")

Predictions saved to /content/drive/MyDrive/DATS 5980/test_predictions_mcq.pkl


In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Wed Sep 17 09:51:48 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   59C    P0             27W /   70W |    3278MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----